# Generating a population of datasets using a genetic algorithm
---

## Motivation
---
Preliminary tests on relatively small datasets has shown that deliberately choosing a fairer starting point for the algorithm (i.e. by assigning potential centroids to points in the data by solving a matching game) does not necessarily improve the performance of the overall $k$-modes algorithm.

The hope is to develop a set of datasets which are better clustered by our matching initialisation than by Cao's method (the leading competitor). We will develop this set as a population in a genetic algorithm.

## Method
---
#### 1. Beginning with an initial set of randomly-generated categorical datasets
 - Defined by their number of **rows, columns, and clusters** we will cluster them each by our method and Cao's.

#### 2. Select two parents that are well-performing
 - Performance is given by our **objective function**
 - We stipulate some level of **difference** between these parent datasets to **maintain** some population **diversity**. By doing this, we avoid falling into a local minimum too soon.

#### 3. Create an offspring based on some crossover operation
 - This will likely be some mix of blending the columns of our parent datasets, and resizing it to be full.

#### 4. Select some underperforming dataset and replace it with the offspring. Go to 2 until some stopping criterion is met


At each generation, we 'roll a dice' and mutate a dataset at random according to some pre-defined mutation rate. By mutating a member of our population, we force some diversity even when we are converging to some 'stable' population of datasets. This mutation operation will be similar to the crossover operation.

In [3]:
from kmodes.kmodes import KModes
from sklearn.datasets import make_blobs

import operator

import pandas as pd
import numpy as np

In [4]:
def pointwise_dissim(x, y):
    return np.sum(x != y, axis=0)

def dissim(Y, x):
    return np.sum(Y != x, axis=1)

In [6]:
class DataSet(object):
    """ A dataset object, defined by its number of rows, columns and clusters, and a generator seed. """

    def __init__(self, n_rows, n_cols, n_clusters, seed):
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.n_clusters = n_clusters
        self.seed = seed

        np.random.seed(self.seed)
        self.cluster_std = (0.5 - 0.01) * np.random.random() + 0.01

    def get_dataframe(self):
        """ Generate the dataset itself as a pandas.DataFrame object. """

        data, target = make_blobs(self.n_rows, self.n_cols,
                                  self.n_clusters, self.cluster_std,
                                  center_box=(0, 1), random_state=self.seed)

        data = np.round(data, 0)
        dataframe = pd.DataFrame({f'attr{col}': data[:, col] for col in range(data.shape[1])})

        return dataframe

    def get_clustering(self, init):
        """ Cluster the dataset into `n_clusters` parts, initialised by method `init`. """

        km = KModes(n_clusters=self.n_clusters, init=init, n_init=10)
        km.fit_predict(self.get_dataframe)

        return km

    def fitness(self):
        """ Find the fitness of a dataset by clustering it by Cao's method, and our own;
        return the difference between their costs. """

        cao = self.get_clustering('cao')
        matching_best = self.get_clustering('matching_best')

        return matching_best.cost_ - cao.cost_

In [23]:
def generateFirstPopulation(population_size):

    population = []
    seed = 0

    while seed < population_size:

        np.random.seed(seed)
        n_rows = np.random.randint(100, 10000)
        n_cols = np.random.randint(4, 500)
        n_clusters = np.random.randint(3, 20)

        while n_clusters >= n_rows:
            n_clusters = np.random.randint(min(n_rows, 3), 20)

        dataset = DataSet(n_rows, n_cols, n_clusters, seed)
        population.append(dataset)
        seed += 1

    return population

In [24]:
def getOrderedPopulation(population):

    ordered_population = {}
    for individual in population:
        ordered_population[individual] = individual.fitness()

    return sorted(ordered_population.items(), key=operator.itemgetter(1), reverse=True)

In [25]:
population = generateFirstPopulation(5)

In [30]:
for ind in population:
    print(ind.n_clusters, ind.get_dataset().shape)

3 (2832, 51)
11 (335, 400)
16 (7436, 19)
6 (6094, 156)
8 (1246, 178)


In [26]:
getOrderedPopulation(population)

AssertionError: Cannot have more clusters (3) than data points (1).